In [82]:
import argparse 
import datetime
import random

import numpy as np
import tensorflow as tf


args = argparse.Namespace()
args.seed = 101
args.epochs = 20
args.batch_size = 32
    
def random_date_output():
    ordinal_min = datetime.date(1000, 1, 1).toordinal()
    ordinal_max = datetime.date(9999, 12, 31).toordinal()
    return datetime.date.fromordinal(random.randint(ordinal_min, ordinal_max))

def output_date_to_input(date) -> str:
    months = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]
    
    return "{} {}, {}".format(months[date.month - 1], date.day, date.year)


def date_to_digits(str_date):
    str_date = str_date.lower()
    return [ord(s) for s in str_date]

def output_date_to_digits(str_date):
    return [10 if s=='-' else int(s) for s in str_date]

def input_tensor_to_date(np_date) -> str:
    date_str = "".join([chr(i) for i in np_date])
    return date_str[0].upper() + date_str[1:]

def output_tensor_to_date(tensor_date) -> str:
    return "".join(["-" if i==10 else str(i.numpy()) for i in tensor_date])

def prediction_to_output_tensor(prediction, axis=1):
    return tf.argmax(prediction, axis=axis, output_type=tf.int32)

def build_dataset(dates_num):
    input_list = []
    output_list = []
    for i in range(dates_num):
        rd = random_date_output()
        ird = output_date_to_input(rd)
        ird = date_to_digits(ird)
        rd = output_date_to_digits(str(rd))
        input_list.append(ird)
        output_list.append(rd)
    return tf.ragged.constant(input_list), tf.constant(output_list)


train_dataset = build_dataset(20000)
val_dataset = build_dataset(10000)

in_num = ord("z")
out_num = 11
max_out_len = len(train_dataset[1][0])

In [3]:
def simple_enc_dec():    
    encoder = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(in_num, 32, input_shape=[None]),
        tf.keras.layers.LSTM(64, return_sequences=False)])

    # RepeatVector is a must. we translate +- 17 words seq into 10 words seq. return_sequences = True for encoder leads up to 17 words output
    # with return_sequences = False, it won't be possible to reconstruct a full seq, but one letter.
    decoder = tf.keras.models.Sequential([
        tf.keras.layers.RepeatVector(max_out_len),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(out_num, activation="softmax")])
    model = tf.keras.models.Sequential([encoder, decoder])

    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    monitor='val_loss'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=3, mode='auto', restore_best_weights=True, verbose=1)
    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=2, min_delta=1e-4, mode='auto', verbose=1)
    encoder.summary()
    decoder.summary()
    model.summary()
    model.fit(train_dataset[0], train_dataset[1], batch_size=args.batch_size, epochs=args.epochs, validation_data=(val_dataset[0], val_dataset[1]), callbacks=[early_stopping, reduce_lr_on_plateau])

    for i in range(5):
        print("***")
        print(val_dataset[0][i])
        print(input_tensor_to_date(val_dataset[0][i]))
        prediction = model(val_dataset[0])[i]
        output_tensor = prediction_to_output_tensor(prediction)
        print(output_tensor)
        print(output_tensor_to_date(output_tensor))
    
if True:
    simple_enc_dec()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          3904      
                                                                 
 lstm_2 (LSTM)               (None, 64)                24832     
                                                                 
Total params: 28,736
Trainable params: 28,736
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 repeat_vector_1 (RepeatVect  (None, 10, 64)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 10, 64)            33024     
                            

/Users/mkhokhlush/github/ml-experiments/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_5/sequential_3/lstm_2/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_5/sequential_3/lstm_2/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_5/sequential_3/lstm_2/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


625/625 [==============================] - 10s 11ms/step - loss: 1.5200 - accuracy: 0.4776 - val_loss: 0.9695 - val_accuracy: 0.6598 - lr: 0.0010
Epoch 2/20
625/625 [==============================] - 6s 10ms/step - loss: 0.7871 - accuracy: 0.7266 - val_loss: 0.5666 - val_accuracy: 0.8049 - lr: 0.0010
Epoch 3/20
625/625 [==============================] - 6s 10ms/step - loss: 0.4372 - accuracy: 0.8481 - val_loss: 0.3336 - val_accuracy: 0.8855 - lr: 0.0010
Epoch 4/20
625/625 [==============================] - 6s 10ms/step - loss: 0.3237 - accuracy: 0.8968 - val_loss: 0.2013 - val_accuracy: 0.9414 - lr: 0.0010
Epoch 5/20
625/625 [==============================] - 6s 10ms/step - loss: 0.1567 - accuracy: 0.9588 - val_loss: 0.1139 - val_accuracy: 0.9732 - lr: 0.0010
Epoch 6/20
625/625 [==============================] - 6s 10ms/step - loss: 0.1651 - accuracy: 0.9604 - val_loss: 0.1001 - val_accuracy: 0.9781 - lr: 0.0010
Epoch 7/20
625/625 [==============================] - 6s 10ms/step - loss:

In [101]:
def enc_shifted_dec():
    sos = out_num # 10 digits + "-" = [0, 10] => "sos" = 11
    def build_decoder_inputs(targets):
        soss = tf.fill([targets.shape[0], 1], sos) # targets.shape == decoder_inputs
        return tf.concat([soss, targets[:, :-1]], axis=1) # need only previous states for the targets in the decoder in -> no last char
    train_decoder_inputs = build_decoder_inputs(train_dataset[1])
    val_decoder_inputs = build_decoder_inputs(val_dataset[1])
    
    enc_in = tf.keras.layers.Input([None], ragged=True)
    enc_emb = tf.keras.layers.Embedding(in_num, 32)(enc_in)
    enc_out, enc_m_state, enc_c_state = tf.keras.layers.LSTM(64, return_sequences=False, return_state=True)(enc_emb)
    
    dec_in = tf.keras.layers.Input([max_out_len])
    dec_emb = tf.keras.layers.Embedding(sos + 1, 4)(dec_in)
    dec_lstm = tf.keras.layers.LSTM(64, return_sequences=True)(dec_emb, initial_state=[enc_m_state, enc_c_state])
    dec_out = tf.keras.layers.Dense(out_num, activation="softmax")(dec_lstm)
    
    model = tf.keras.Model(inputs=[enc_in, dec_in], outputs=[dec_out])
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    monitor='val_loss'
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=3, mode='auto', restore_best_weights=True, verbose=1)
    reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor=monitor, factor=0.1, patience=2, min_delta=1e-4, mode='auto', verbose=1)
    model.fit([train_dataset[0], train_decoder_inputs], train_dataset[1], batch_size=args.batch_size, epochs=args.epochs, validation_data=([val_dataset[0], val_decoder_inputs], val_dataset[1]), callbacks=[early_stopping, reduce_lr_on_plateau])
    
    def try_predict(num=5):
        x = val_dataset[0][:num][:]
        for i in range(num):  
            print(input_tensor_to_date(x[i]))
        x_sos = tf.fill([x.shape[0], 1], 11)
        y_pred = tf.zeros([x.shape[0], 0], dtype=tf.int32)
        y = tf.zeros([x.shape[0], 0], dtype=tf.int32)
        for i in range(max_out_len):
            x_dec = tf.concat([x_sos, y_pred[:, :i], tf.zeros([x.shape[0], max_out_len - i - 1], dtype=tf.int32)], axis=1)
            prediction = model.predict([x, x_dec])
            y_pred = prediction_to_output_tensor(prediction, axis=2)
            y = tf.concat([y, tf.reshape(y_pred[:, i], [-1, 1])], axis=1) # indexing one column creates a vector
            
        assert(tf.reduce_all(tf.equal(y_pred, y)))
        print(y)
        for i in range(num):  
            print(output_tensor_to_date(y[i]))
            
    try_predict()
    return model
    
if True:
    enc_shifted_dec()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_45 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_46 (InputLayer)          [(None, 10)]         0           []                               
                                                                                                  
 embedding_31 (Embedding)       (None, None, 32)     3904        ['input_45[0][0]']               
                                                                                                  
 embedding_32 (Embedding)       (None, 10, 4)        48          ['input_46[0][0]']               
                                                                                           

/Users/mkhokhlush/github/ml-experiments/.venv/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_10/lstm_30/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_10/lstm_30/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/model_10/lstm_30/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


625/625 [==============================] - 10s 11ms/step - loss: 1.3801 - accuracy: 0.5077 - val_loss: 0.9215 - val_accuracy: 0.6543 - lr: 0.0010
Epoch 2/20
625/625 [==============================] - 6s 10ms/step - loss: 0.6137 - accuracy: 0.7908 - val_loss: 0.3570 - val_accuracy: 0.9015 - lr: 0.0010
Epoch 3/20
625/625 [==============================] - 6s 10ms/step - loss: 0.2141 - accuracy: 0.9510 - val_loss: 0.7276 - val_accuracy: 0.7599 - lr: 0.0010
Epoch 4/20
625/625 [==============================] - 6s 10ms/step - loss: 0.0916 - accuracy: 0.9876 - val_loss: 0.0520 - val_accuracy: 0.9965 - lr: 0.0010
Epoch 5/20
625/625 [==============================] - 6s 10ms/step - loss: 0.0449 - accuracy: 0.9962 - val_loss: 0.0278 - val_accuracy: 0.9991 - lr: 0.0010
Epoch 6/20
625/625 [==============================] - 6s 10ms/step - loss: 0.0208 - accuracy: 0.9996 - val_loss: 0.0159 - val_accuracy: 0.9998 - lr: 0.0010
Epoch 7/20
625/625 [==============================] - 6s 10ms/step - loss: